In [25]:
import numpy as np
import pandas as pd

In [26]:
embed=pd.read_csv("dataset/train_embed.csv")
embed_train=pd.read_csv("dataset/train_embed1.csv")

In [27]:
embed.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.426163,-0.019715,-0.147427,-0.191711,-0.247835,-0.007556,0.184607,0.230126,-0.112902,-0.135185,...,0.208031,-0.077033,0.154945,-0.392426,0.158891,-0.087199,0.057379,-0.149604,0.227732,0.445528
1,-0.295872,-0.050357,-0.142741,-0.081953,-0.169617,-0.029728,0.289099,0.332459,-0.120551,-0.146456,...,0.179806,-0.021631,0.080972,-0.229730,0.016145,-0.030169,-0.030471,-0.201758,0.298551,0.430040
2,-0.265745,0.226682,-0.218188,-0.155952,-0.164947,-0.177518,0.537117,0.106359,-0.226721,-0.027696,...,0.069250,-0.303264,0.013453,-0.468215,0.262105,0.055382,-0.105407,0.006217,0.484209,0.536531
3,-0.527993,-0.099281,-0.097664,-0.083705,0.137262,-0.064075,-0.087627,0.044312,-0.224528,-0.119273,...,0.342399,-0.212913,-0.095079,-0.352270,-0.041175,0.016697,0.025686,0.047509,0.133078,0.158953
4,-0.430770,-0.092300,-0.105847,-0.048876,-0.134100,-0.038064,0.030199,0.198923,-0.104836,-0.186939,...,0.129804,-0.015920,0.102030,-0.447980,0.006463,-0.048333,-0.100819,-0.153923,0.216303,0.485919


In [28]:
embed_train.head()

,0
0,492.125984
1,500.000000
2,2.000000
3,3150.000000
4,700.000000


In [29]:
# embed["PRODUCT_ID"]=embed_train[0]

In [30]:
def average(pred):
    return pred.sum()/pred.shape[0]

In [31]:
import numpy as np
import faiss
import time

class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        self.y = y

    def predict(self, X):
        st=time.time()
        distances, indices = self.index.search(X.astype(np.float32), k=self.k)
        print("Time Elapsed = ",time.time()-st)
        votes = self.y[indices]
        predictions = np.array([average(x) for x in votes])
        return predictions

In [32]:
X = np.array(embed)
Y = np.array(embed_train)

In [33]:
print(X.shape, Y.shape)

(300000, 768) (300000, 1)


In [34]:
nr=FaissKNeighbors()

In [35]:
nr.fit(X,Y)

In [36]:
x_test=np.array(pd.read_csv("dataset/test_embed.csv"))

In [46]:
y_test=np.array(pd.read_csv('dataset/test_embed1.csv'))

In [38]:
pred=nr.predict(x_test)

Time Elapsed =  245.5104055404663


In [39]:
sub = pd.read_csv('dataset/sample_submission.csv')

In [40]:
len(pred)

734736

In [47]:
print(y_test)

[[ 6142.]
 [ 1622.]
 [ 7540.]
 ...
 [10645.]
 [12680.]
 [    0.]]


In [44]:
print(sub)

        PRODUCT_ID  PRODUCT_LENGTH
0           604373      687.400200
1          1729783     6379.999999
2          1871949      880.000000
3          1107571      227.559041
4           624253      718.834400
...            ...             ...
734731      921419      100.000000
734732     2456362      650.000000
734733      841529      935.984252
734734     1190194     1080.000000
734735     1040810     1334.100000

[734736 rows x 2 columns]


In [43]:
sub['PRODUCT_LENGTH'] = pred

In [45]:
sub.to_csv('last_sub.csv', index=False)